In [1]:
pip install opencv-python dlib

In [2]:
import cv2
import dlib
import numpy as np

In [3]:
#load models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [5]:
import cv2
import dlib
import numpy as np

# Initialize detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Helper: calculate angle from nose bridge
def get_head_tilt_direction(shape):
    # Points on the nose bridge (27 and 30)
    nose_top = shape.part(27)
    nose_tip = shape.part(30)

    # Calculate the vertical movement
    dy = nose_tip.y - nose_top.y

    # NEW: Flip logic to fix inverted output
    if dy > 12:  # More downward means head tilted DOWN
        return "Head Down"
    elif dy < 6:  # Less vertical distance means head tilted UP
        return "Head Up"
    else:
        return "Head Straight"

# Start video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # MODIFIED: Preprocess to help with poor lighting/contrast
    gray = cv2.equalizeHist(gray)

    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)

        # Draw face rectangle
        cv2.rectangle(frame, (face.left(), face.top()), (face.right(), face.bottom()), (0, 255, 0), 2)

        # Draw all landmarks
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            cv2.circle(frame, (x, y), 1, (255, 0, 0), -1)

        # Check head direction
        direction = get_head_tilt_direction(landmarks)
        cv2.putText(frame, direction, (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # MODIFIED: Add fallback text if no face is detected
    if len(faces) == 0:
        cv2.putText(frame, "Face not detected", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Head Tilt Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [4]:
# #CODE TO DETECT HEAD TILT USING DLIB
# import cv2
# import dlib
# import numpy as np

# # Load detector and predictor
# detector = dlib.get_frontal_face_detector()
# predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# # 3D model points (generic model)
# model_points = np.array([
#     (0.0, 0.0, 0.0),             # Nose tip
#     (0.0, -330.0, -65.0),        # Chin
#     (-225.0, 170.0, -135.0),     # Left eye left corner
#     (225.0, 170.0, -135.0),      # Right eye right corner
#     (-150.0, -150.0, -125.0),    # Left mouth corner
#     (150.0, -150.0, -125.0)      # Right mouth corner
# ])

# # Start video
# cap = cv2.VideoCapture(0)

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     faces = detector(gray)

#     for face in faces:
#         shape = predictor(gray, face)
#         landmarks = np.array([[p.x, p.y] for p in shape.parts()])

#         image_points = np.array([
#             landmarks[30],  # Nose tip
#             landmarks[8],   # Chin
#             landmarks[36],  # Left eye left corner
#             landmarks[45],  # Right eye right corner
#             landmarks[48],  # Left mouth corner
#             landmarks[54]   # Right mouth corner
#         ], dtype="double")

#         # Camera internals (assuming webcam with default values)
#         size = frame.shape
#         focal_length = size[1]
#         center = (size[1] / 2, size[0] / 2)
#         camera_matrix = np.array([
#             [focal_length, 0, center[0]],
#             [0, focal_length, center[1]],
#             [0, 0, 1]
#         ], dtype="double")

#         dist_coeffs = np.zeros((4, 1))  # Assuming no lens distortion
#         success, rotation_vector, translation_vector = cv2.solvePnP(
#             model_points, image_points, camera_matrix, dist_coeffs
#         )

#         # Convert rotation vector to Euler angles
#         rmat, _ = cv2.Rodrigues(rotation_vector)
#         angles, _, _, _, _, _ = cv2.RQDecomp3x3(rmat)

#         pitch = angles[0]  # Up/down tilt
#         yaw = angles[1]    # Left/right turn
#         roll = angles[2]   # Side tilt

#         # Display tilt status
#         if pitch > 15:
#             cv2.putText(frame, "Head tilted DOWN", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
#         elif pitch < -15:
#             cv2.putText(frame, "Head tilted UP", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

#         cv2.putText(frame, f"Pitch: {int(pitch)}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

#     cv2.imshow("Head Pose Estimation", frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()
